In [1]:
import presto
import pandas as pd
import numpy as np
import pulsar
import os
import plotly.express as px
import plotly.graph_objects as go

In [103]:
def presto_query(query, user='test', catalog='pulsar', schema='public/default', host='localhost', port=8081):
    conn = presto.dbapi.connect(
        host=host,
        port=port,
        user=user,
        catalog=catalog,
        schema=schema)
    cur = conn.cursor()
    cur.execute(query)
    data = cur.fetchall()
    columns = cur.description
    df = pd.DataFrame(data)
    df.columns = [x[0] for x in columns]
    return df.replace('', np.nan)

In [138]:
df = presto_query("SELECT * FROM last_balance", schema='public/default')
df.tail()

,balance,behaviorprob,entitytype,namespace,numjobs,simnum,user,__partition__,__event_time__,__publish_time__,__message_id__,__sequence_id__,__producer_name__,__key__,__properties__
48,1059.9366,0.00,customer,sim-8,999,8,c-sim-8-0,-1,None,2020-10-22 18:41:10.233,"(347,48,0)",0,standalone-0-498,None,{}
49,1064.8165,0.25,customer,sim-8,999,8,c-sim-8-1,-1,None,2020-10-22 18:41:11.000,"(347,49,0)",0,standalone-0-499,None,{}
50,1070.3962,0.50,customer,sim-8,999,8,c-sim-8-2,-1,None,2020-10-22 18:41:11.676,"(347,50,0)",0,standalone-0-500,None,{}
51,1074.9160,0.75,customer,sim-8,999,8,c-sim-8-3,-1,None,2020-10-22 18:41:11.715,"(347,51,0)",0,standalone-0-501,None,{}
52,1079.9159,1.10,customer,sim-8,999,8,c-sim-8-5,-1,None,2020-10-22 18:41:11.906,"(347,52,0)",0,standalone-0-502,None,{}


In [137]:
SUPPLIER_PROBS = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
CUSTOMER_PROBS = [0.0, 0.25, 0.5, 0.75, 1.0]

df_plot = {f"{prob}":[] for prob in CUSTOMER_PROBS}
df_plot['index'] = []
for i in sorted(df['simnum'].unique().tolist()):
    df_plot['index'].append(i)
    
    temp = df[df['simnum']==i]
    sorted_props = sorted(df['behaviorprob'].unique().tolist())
    for behaviorprob in sorted_props:
        t = temp.loc[temp['behaviorprob']==behaviorprob, 'balance'].values.tolist()
        try:
            df_plot[f"{behaviorprob}"].append(t[0]-1000)
        except:
            pass
df_plot

{'0.0': [-102.60329999999999,
  -90.00329999999997,
  -62.70330000000001,
  -40.86329999999998,
  -17.76329999999996,
  -6.003299999999967,
  16.676699999999983,
  35.99669999999992,
  59.9366],
 '0.25': [-96.06349999999998,
  -81.48350000000005,
  -55.24350000000004,
  -42.50350000000003,
  -15.543499999999995,
  7.816469999999981,
  28.856399999999894,
  45.67650000000003,
  64.8164999999999],
 '0.5': [-93.30370000000005,
  -76.42370000000005,
  -55.74369999999999,
  -32.92370000000005,
  -12.223700000000008,
  2.9963000000000193,
  20.116300000000024,
  50.756200000000035,
  70.39619999999991],
 '0.75': [-94.34393,
  -67.04395,
  -41.94399999999996,
  -28.023900000000026,
  -10.303959999999961,
  15.67610000000002,
  33.55600000000004,
  55.09609999999998,
  74.91599999999994],
 '1.0': [-97.32416,
  -57.844100000000026,
  -46.084100000000035,
  -27.184139999999957,
  -4.924129999999991,
  20.27589999999998,
  34.975799999999936,
  55.975799999999936],
 'index': [0, 1, 2, 3, 4, 5, 6,

In [139]:
df_plot['1.0'].append(79.9)

In [140]:
df_plot = pd.DataFrame(df_plot)
df_plot['x'] = SUPPLIER_PROBS
df_plot = df_plot.rename(columns={"0.0": "a", "0.25": "b", "0.5": "c", "0.75": "d", "1.0": "e"}, errors="raise")
df_plot.head()

,a,b,c,d,e,index,x
0,-102.6033,-96.0635,-93.3037,-94.34393,-97.32416,0,0.60
1,-90.0033,-81.4835,-76.4237,-67.04395,-57.84410,1,0.65
2,-62.7033,-55.2435,-55.7437,-41.94400,-46.08410,2,0.70
3,-40.8633,-42.5035,-32.9237,-28.02390,-27.18414,3,0.75
4,-17.7633,-15.5435,-12.2237,-10.30396,-4.92413,4,0.80


In [141]:
df_plot.to_csv(os.path.join(os.getcwd(), "data", "customersimresults1000-2.csv"), index=False)

In [100]:
df_plot.tail()

,a,b,c,d,e,index,x
3,-113.5233,-112.2235,-94.54376,-98.22394,-99.42413,3,0.60
4,-13.9833,-13.6435,-20.80370,-7.08390,-1.56415,4,0.80
5,17.5167,31.6365,29.75620,30.25610,36.23580,5,0.90
6,34.7367,39.4564,51.57630,52.43600,53.03590,6,0.95
7,59.9366,65.0365,70.53630,75.09610,79.91590,7,1.00
